In [23]:
from database.connect import getConnection
import mysql
import pandas as pd
from pathlib import Path
import csv

try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

Runtime W df method: 5.4 s  
Runtime W str method: 5.7 s

In [24]:
file_path = Path("./pre-processed/wordcounts.csv")

# check if the file exists
my_file = file_path
if not my_file.is_file():
    import nltk
    from nltk.tokenize import word_tokenize
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')

    incluided_tags = ['JJ', 'NN', 'RP', 'VB']
    # Adjective, noun, particle, verb

    word_counts = pd.DataFrame(columns=["word", "count"])

    # get the text from the database
    cursor.execute("SELECT text FROM tweets WHERE text not like 'RT%' AND language = 'en'")
    tweets = cursor.fetchall()
    print('got all tweets')

    word_counts = {}
    for tweet in tweets:
        tokenized_tweet = word_tokenize(tweet[0])
        tweet_pos = nltk.pos_tag(tokenized_tweet)

        words = [word for word, pos in tweet_pos if pos[:2] in incluided_tags]

        for word in words:
            word = word.lower().strip(".,:;!?\"\'()[]{}")
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1

    # create the file if it doesn't exist
    with open(file_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["word", "count"])
        for word, count in sorted(word_counts.items(), key=lambda x: x[1], reverse=True):
            writer.writerow([word, count])
       

In [25]:
categories = {'booking': ['booking', 'booked', 'book', 'ticket', 'tickets'],
            'canceling': ['canceled', 'cancellations'],
            'money': ['refund', 'compensation', 'claim', 'money', 'pay', 'paid'],
            'baggage': ['bag', 'baggage', 'luggage', 'bags'],
            'staff': ['staff', 'crew'],
            'waiting': ['waiting', 'delay', 'wait'],
            'boarding': ['boarding'],
            'stuck': ['stuck'],

            'information': ['info', 'information'],
            'customers': ['customer', 'customers', 'passenger', 'passengers'],
            'dm': [' dm'], 
            }

In [26]:
# test if the tweets table has a column for any of the categories
category_names = set(categories.keys())
query = "SHOW COLUMNS FROM tweets"
cursor.execute(query)
columns = cursor.fetchall()
column_names = set([column[0] for column in columns])
# check if there are any columns with the category names
columns_without_categories = column_names - category_names
if columns_without_categories != columns:
    for category in category_names:
        # add the column
        query = f"ALTER TABLE tweets ADD {category} INT(1) DEFAULT 0"
        cursor.execute(query)
        
        # get the words for the category
        words = categories[category]

        query = f"UPDATE tweets SET {category} = 1 WHERE text REGEXP '{'|'.join(words)}'"
        cursor.execute(query)
        connection.commit()
        print(f'Added column {category} to tweets table')

Added column baggage to tweets table
Added column waiting to tweets table
Added column staff to tweets table
Added column dm to tweets table
Added column stuck to tweets table
Added column canceling to tweets table
Added column booking to tweets table
Added column customers to tweets table
Added column money to tweets table
Added column information to tweets table
Added column boarding to tweets table
